In [1]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 40.1 MB/s eta 0:00:00


In [2]:
import time
import requests
import pandas as pd
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output

# API
ISS_API = "http://api.open-notify.org/iss-now.json"
UPDATE_INTERVAL_MS = 5000
TRAIL_LENGTH = 100

# Fetch ISS Position

def fetch_iss_position():
    try:
        r = requests.get(ISS_API, timeout = 10)
        data = r.json()
        lat = float(data['iss_position']['latitude'])
        lon = float(data['iss_position']['longitude'])
        ts = int(data.get("timestamp", time.time()))

        return lat, lon, ts

    except Exception as e:
        print("Error:", e)

# Initial Figure
fig = go.Figure()

# Dash App

app = Dash(__name__)

app.layout = html.Div([html.H3("Real-time ISS Tracking"),
                       dcc.Graph(id = "map", figure = fig, style = {"height": "85vh"}),
                       dcc.Interval(id = "interval", interval = UPDATE_INTERVAL_MS, n_intervals = 0),
                       html.Div(id = "status", style = {"padding":"8px", "fontSize":"14px"})])

# Store ISS trail

trail_df = pd.DataFrame(columns = ["ts", "lat", "lon"])

@app.callback(Output("map", "figure"),
              Output("status", "children"),
              Input("interval", "n_intervals"))

def update_map(n):

    global trail_df, fig

    pos = fetch_iss_position()

    if pos is None:
        return fig, "Failed to fetch ISS Position"

    lat, lon, ts = pos

    new_row = {"ts":ts, "lat":lat, "lon":lon}

    trail_df = pd.concat([trail_df, pd.DataFrame([new_row])], ignore_index= True)

    if len(trail_df) > TRAIL_LENGTH:
        trail_df = trail_df.iloc[-TRAIL_LENGTH:]

    fig = go.Figure()

    # Trail
    fig.add_trace(go.Scattergeo(lon = trail_df["lon"],
                                lat = trail_df["lat"],
                                mode = "lines",
                                line = {"width":2, "color":"orange"},
                                name = "Trail"))

    # ISS marker

    fig.add_trace(go.Scattergeo(lon = [lon],
                                lat = [lat],
                                mode = "markers",
                                marker = {"size":8, "color": "red"},
                                name = "ISS"))

    fig.update_geos(projection_type = "natural earth",
                    showcoastlines = True, coastlinecolor = "black",
                    showland = True, landcolor = "rgb(230, 230, 230)",
                    showocean = True, oceancolor = "rgb(230, 240, 250)")

    fig.update_layout(title = "Real-time ISS Tracker",
                      margin = {"r":0, "l":0, "t":40, "b":0})

    status = f"Updated:{pd.to_datetime(ts, unit = 's')} | Lat:{lat:.4f}, Lon: {lon:.4f}"

    return fig, status

if __name__ == "__main__":
    app.run(debug= True)


<IPython.core.display.Javascript object>